In [2]:
import requests
from bs4 import BeautifulSoup
import re
import random
import pandas as pd
import time
from tqdm import tqdm     
import json
with open('./hidak_links.json', 'r') as f:
    links_list = json.load(f)

list1 = links_list['소아청소년과']

def scrape_hidoc_data(list1):
    head = { 'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
    
    data = []
    
    for link in tqdm(list1):
        url = f"https://mobile.hidoc.co.kr/healthqna/part/{link}"
        r = requests.get(url, headers=head)
        bs = BeautifulSoup(r.text, 'html.parser')

        date_data = bs.find("span", class_='date')
        date = date_data.text

        title_data = bs.select('.tit_qna')
        pattern = r'<strong class="tit_qna">\s*(.*?)\s*<div class="qna_info">'

        title_data_ = str(title_data[0])

        title = re.search(pattern, title_data_)

        if title:
            title_ = title.group(1)
        else:
            title_ = "매칭되는 부분이 없습니다."

        question_data = bs.find('div', class_='desc')   
        question = question_data.text.strip() 

        doctor_info = bs.find_all("strong", class_='link_doctor')

        doctor_list = []
        for x in range(len(doctor_info)):
            doctor_list.append(doctor_info[0].text)


        doc_hospital = bs.find_all("span", class_ ='txt_clinic')

        hospital_list = []
        for x in range(len(doc_hospital)):
            hospital_list.append(doc_hospital[0].text)

        a = bs.findAll("div", class_ = 'desc')
        answer = []
        for x in range(1,len(a)):
            answer.append(a[x].text.strip())

        data.append({
            'Date': date,
            'Title': title_,
            'Question': question,
            'Doctors': doctor_list,
            'Hospitals': hospital_list,
            'Answers': answer
        })
    
    df = pd.DataFrame(data)
    return df


# 함수 호출
df = scrape_hidoc_data(list1)


100%|██████████| 6469/6469 [54:53<00:00,  1.96it/s]  


In [4]:
df.to_csv("./소아청소년과_QNA.csv", index = False, encoding = 'utf-8')